# Latitude-dependent grey radiation

Here is a quick example of using the `climlab.GreyRadiationModel` with a latitude dimension and seasonally varying insolation.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import climlab

In [2]:
np.__version__

'1.8.1'

In [3]:
model = climlab.GreyRadiationModel(num_lev=30, num_lat=90)
print model

ValueError: operands could not be broadcast together with shapes (90,31) (90,31,31) 

In [3]:
np.__version__

'1.8.1'

In [ ]:
print model.lat

In [ ]:
insolation = climlab.radiation.insolation.DailyInsolation(domains=model.Ts.domain)

In [ ]:
model.add_subprocess('insolation', insolation)

In [ ]:
print model

In [ ]:
model.step_forward()

In [ ]:
plt.plot(model.lat, model.diagnostics['insolation'])

In [ ]:
model.Tatm.shape

In [ ]:
model.integrate_years(1)

In [ ]:
plt.plot(model.lat, model.Ts)

In [ ]:
model.integrate_years(1)

In [ ]:
plt.plot(model.lat, model.timeave['Ts'])

In [ ]:
def plot_temp_section(model, timeave=True):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    if timeave:
        field = model.timeave['Tatm'].transpose()
    else:
        field = model.Tatm.transpose()
    cax = ax.contourf(model.lat, model.lev, field)
    ax.invert_yaxis()
    ax.set_xlim(-90,90)
    ax.set_xticks([-90, -60, -30, 0, 30, 60, 90])
    fig.colorbar(cax)

In [ ]:
plot_temp_section(model)

In [ ]:
model2 = climlab.RadiativeConvectiveModel(num_lev=30, num_lat=90)
insolation = climlab.radiation.insolation.DailyInsolation(domains=model2.Ts.domain)
model2.add_subprocess('insolation', insolation)

In [ ]:
model2.step_forward()

In [ ]:
model2.integrate_years(1)

In [ ]:
model2.integrate_years(1)

In [ ]:
plot_temp_section(model2)

## Testing out multi-dimensional Band Models

In [ ]:
import climlab
import numpy as np

#  Put in some ozone
import netCDF4 as nc

datapath = "http://ramadda.atmos.albany.edu:8080/repository/opendap/latest/Top/Users/Brian+Rose/CESM+runs/"
endstr = "/entry.das"

topo = nc.Dataset( datapath + 'som_input/USGS-gtopo30_1.9x2.5_remap_c050602.nc' + endstr )
ozone = nc.Dataset( datapath + 'som_input/ozone_1.9x2.5_L26_2000clim_c091112.nc' + endstr )

#  Dimensions of the ozone file
lat = ozone.variables['lat'][:]
lon = ozone.variables['lon'][:]
lev = ozone.variables['lev'][:]

# Taking annual, zonal average of the ozone data
O3_zon = np.mean( ozone.variables['O3'],axis=(0,3) )

In [ ]:
import climlab
import numpy as np
#  make a model on the same grid as the ozone
model3 = climlab.BandRCModel(lev=lev, lat=lat)
insolation = climlab.radiation.insolation.DailyInsolation(domains=model3.Ts.domain)
model3.add_subprocess('insolation', insolation)
print model3

In [ ]:
#  Set the ozone mixing ratio
#   IMPORTANT: we need to flip the ozone array around because the vertical coordinate runs the wrong way
#   (first element is top of atmosphere, whereas our model expects the first element to be just above the surface)
O3_trans = np.transpose(O3_zon)
print O3_trans.shape
print lev
print model3.lev


In [ ]:
print O3_trans[0,:]
O3_flipped = np.fliplr(O3_trans)
print O3_flipped[0,:]

In [ ]:
# Let's try not putting in the ozone
#model3.absorber_vmr['O3'] = O3_flipped

In [ ]:
print model3.absorber_vmr['O3'].shape
print model3.Tatm.shape

In [ ]:
model3.integrate_years(1.)

In [ ]:
model3.integrate_years(1.)

In [ ]:
#plt.contour(model3.lat, model3.lev, model3.Tatm.transpose())
plot_temp_section(model3)

This is now working. Will need to do some model tuning.

And start to add dynamics!